In [ ]:
import torch
from torch.utils.data import DataLoader
from datetime import datetime
from utils.data.CountyDataset import (
    CountyDataset,
)
from torch.utils.data import random_split
from torch import nn

In [ ]:
def get_dataset(master_path):
    dataset = CountyDataset(master_path)
    X, Y = next(iter(dataset))

    print('-'*89)
    print('Total Dataset Summary')
    print('-'*89)

    print(f'total samples {len(dataset):,} '
          f'X {X.size()} | Y {Y.size()}')

    return dataset

def get_numpy(dataset):
    data = []
    for i in range(len(dataset)):
        X, Y = dataset[i]
        data.append((X.numpy(), Y.numpy()))

    print('-'*89)
    print(f'Numpy Dataset Summary')
    print('-'*89)
    X, Y = zip(*data)
    print(f'total samples {len(X)} '
          f'| input time periods {len(X[0])} '
          f'| input features {X[0].shape[1]} '
          f'| output time periods {len(Y[0])}'
          f'| target col {dataset.target_col}')

    return data, dataset.xcols, dataset.target_col, dataset.date_index



In [ ]:
### LOAD DATASET FROM CHRIS'S THING ###
master_path = 'utils/data/county_table.csv'
data = get_numpy(get_dataset(master_path))

In [ ]:
### LOAD DATASET FROM PICKLE ###
import pickle
with open('dataset.pickle', 'rb') as handle:
    full_dataset = pickle.load(handle)

In [ ]:
# full_dataset = data[0] ## comment this if loaded from pickle
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset,
                                                            [1267, 317])
train_loader = DataLoader(train_dataset, batch_size=128,
                          shuffle=True, pin_memory=False,
                          num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=128,
                          shuffle=True, pin_memory=False,
                          num_workers=4)



In [ ]:
### TRAINING ###
### PICK A MODEL ###
from models.LSTM import LSTM
model = LSTM(input_dim=235, hidden_dim=10, output_dim=10, num_layers=2)

# from models.RNN import RNN
# model = RNN(D_in=235, H=10, D_out=10, L=1, dropout=0.0)

num_epochs = 10
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for i in range(num_epochs):
    for idx, data in enumerate(train_loader):
        X_train, y_train = data
        
        # zero out gradient between epochs
        optimizer.zero_grad()

        # forward pass
        y_pred = model(X_train)

        # calculate loss
        loss = loss_function(y_pred, y_train)
        if idx == len(train_loader) - 1:
            print("Epoch: ", i, "RMSE: ", torch.sqrt(loss).item())
        # backward pass
        loss.backward()

        # update params
        optimizer.step()
#         print('here')

In [ ]:
### TESTING ###
import numpy as np
with torch.no_grad():
    rmses = []
    for idx, data in enumerate(test_loader):
        X_test, y_test = data
#         X_test = X_test.permute(1, 0, 2)
        y_pred = model(X_test)
        loss = loss_function(y_pred, y_test)
        rmse_i = torch.sqrt(loss).item()
        rmses += [rmse_i]
    print(np.mean(rmses))